# Exploratory Analysis

**Sample Scenario**

We would like to be able to use attributes of customers to estimate their spending score.
In doing so, we can target those customers that are likely to be most profitable.

Our target variable is spending_score.
Currently the only customer data we have available to use in this project is age, annual_income and gender.
It is possible we may not have enough information to build a valuable model. If not, maybe we could do some unsupervised learning, and find clusters of similar customers using all of the variables (including spending_score) and that could help us with a starting point for our targeted marketing.

## Wrangle Data 

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Statistical Tests
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

import wrangle

In [ ]:
min_max_scaler, train, validate, test = wrangle.wrangle_mall_df()

In [ ]:
df = train

In [ ]:
df.head()

In [ ]:
df.describe()

NB. We'll explore unscaled, but scale before modeling.

## Answer Questions

### What is the distribution of each of the variables?

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 7))

for ax, col in zip(axs, df.select_dtypes('number')):
    df[col].plot.hist(ax=ax, title=col, ec='black')

In [ ]:
df.is_male.value_counts().plot.barh()
plt.title('Distribution of Males (1) to Females (0)')
plt.ylabel('is_male')
plt.xlabel('Count')
plt.show()

**Takeaways:**

- spending score is ~ normal
- age + annual income have a long tail on the right -- i.e. they are *right skewed*
- more female observations than male

### Does spending score differ across gender?

1. Viz gender against spending score
1. Stats test to confirm

In [ ]:
sns.violinplot(data=df, y='spending_score', x='is_male')
plt.title('Violin distribution of gender and spending score')
plt.show()

In [ ]:
sns.boxplot(data=df, y='spending_score', x='is_male')
plt.title("Is there a difference in spending score for male vs. female customers?")
plt.show()

In [ ]:
sns.barplot(data=df, y='spending_score', x='is_male')
plt.title("Is there a difference in spending score between males and females?")
plt.show()

**Takeaways:**

- Seems like there's not much difference in spending score.

T-test:

- one-tailed or two-tailed? 2 tailed b/c we are looking for *any* difference in means
- one-sample or two-sample? 2 sample b/c we're looking at the average spending score of 2 separate samples

**Levene's Test**: test for equal variance

- $H_0$: there is no difference in spending score variance between the two samples
- $H_a$: there is a difference in spending score variance between the two samples

In [ ]:
stats.levene(
    df.query('is_male == 1').spending_score,
    df.query('is_male == 0').spending_score,
)

A high pvalue (0.14) means we **fail to reject** the null hypothesis.

In [ ]:
stats.ttest_ind(
    df.query('is_male == 1').spending_score,
    df.query('is_male == 0').spending_score,
    equal_var=True,
)

We conclude there is no significant difference in spending score between Males and Females (p = 0.755).

What does `.query()` do?

In [ ]:
df.query('is_male == 1').head()

In [ ]:
df[df.is_male == 1].head()

**Conclusion:**

### Is there a relationship between spending score and annual income?

1. Viz annual_income by spending_score
2. Spearman's test if we want to confirm correlation (pearson's assumes normally distributed vars)

In [ ]:
df.plot.scatter(
    x="annual_income",
    y="spending_score",
    title='Is there a relationship between annual income and spending score?',
    figsize=(8, 6),
)
plt.show()

**Conclusion:**

- not a linear relationship
- looks like an "X"
- looks like there might be clusters, the middle is very dense, the corners not so much

In [ ]:
df.head()

Is there a relationship between age and spending score?

1. Viz age by spending_score. 
1. Create age bins and compare

In [ ]:
df.plot.scatter(y='spending_score', x='age', title='Is there a relationship between age and spending score?', figsize=(13, 8))
plt.show()

**Takeaways:**

- spending score trails off for older individuals
- younger folks seem to have higher spending scores
- after age ~ 40, max(spending score) decreases

#### How does `pd.cut()` work?

Suppose that we had an array:

In [ ]:
x = pd.Series(range(1, 11))
x

We can pass that array to `pd.cut()` and define the bins with a list of values:

In [ ]:
pd.cut(x, bins=[0, 7, 9, 11])

When working with DataFrames, we can create a bin column quite easily using `pd.cut()`

In [ ]:
df['age_bin'] = pd.cut(df.age, [0, 40, 80])
df.head()

Let's take a look at the distribution of those bins:

In [ ]:
df.age_bin.value_counts().plot.barh(title='Distribution of age_bin')

Looks like there were not any observations with an age above 80:

In [ ]:
df.age.describe()

Let's create a simple boxplot to visualize how spending score compares between these age bins:

In [ ]:
sns.boxplot(data=df, y='spending_score', x='age_bin')
plt.show()

In [ ]:
#H0: Variances are equal
#Ha: Variances are not equal

stats.levene(
    df[df.age <= 40].spending_score,
    df[df.age > 40].spending_score,
)

**Conclusion:**

Our p-value is less than our alpha, so we reject the null hypothesis that the variance in spending score is the same for folks less than or equal to 40 and above 40.

(i.e. the variance is not equal)

In [ ]:
stats.ttest_ind(
    df[df.age <= 40].spending_score,
    df[df.age > 40].spending_score,
    equal_var=False,
)

**Conclusion:**

We reject the null hypothesis that the average spending score for folks less than or equal to 40 is the same as the average spending score for folks over 40.

In [ ]:
df.head()

Note that when we used `pd.cut()` we created a category object. This dtype follows its own set of rules.

In [ ]:
df.dtypes

The values shown are not just strings. If we try to match using a string value, its not going to find it:

In [ ]:
df[df.age_bin == '(0, 40]']

Instead we have to use `pd.Interval()`:

In [ ]:
df[df.age_bin == pd.Interval(0, 40)]

But if we still have access to the original unbinned data, then its probably just more straightforward to use that:

In [ ]:
df[df.age <= 40]

### If we control for age, does spending score differ across annual income?

1. Viz annual income by spending score for each age bin
2. Analyze each age bin separately

In [ ]:
sns.relplot(data=df, y='spending_score', x='annual_income', col='age_bin')

**Takeaways:**

- 0 through 40 group still has an x-shape
- 40+ crowd is just the bottom half of the x

In [ ]:
sns.relplot(
    data=df,
    y='spending_score',
    x='annual_income',
    col=pd.cut(df.age, bins=[0, 30, 40, 80]),
)
plt.suptitle("Do the different decades account for the upper vs lower extremes?")
plt.tight_layout()

**Takeaways:**

- 30-40 *almost* looks linear
- 0-30 *almost* looks like a negative linear relationship
- 40-80 looks quadratic

In [ ]:
df.head()

### Does gender explain the high income-low spending score and/or low income-low spending score in the under 40 group?

In [ ]:
sns.scatterplot(
    x='annual_income',
    y='spending_score',
    data=df[df.age <= 40],
    hue='is_male',
)
plt.title("Does gender acccount for upper vs lower\nin the younger age group?")

Aside: scatterplot vs relplot

- scatter plot works with axes
- relplot works with figures

---

- seaborn is built on top of matplotlib
- every matplotlib axis lives within a figure
- a figure can have 1 or more axes in it (2+ is when we have subplots)

In [ ]:
df.age_bin.value_counts()

In [ ]:
sns.relplot(
    x='annual_income',
    y='spending_score',
    data=df,
    hue='is_male',
    col='age_bin',
)
print("Does gender acccount for upper vs lower in the younger age group?\n")
plt.show()

**Takeaways:**

- gender isn't terribly informative in this context

In [ ]:
df.head()

### If we control for annual income, does spending score differ across age?

Because of the shape of annual income with spending score, I will create 3 bins of income: \[0, 40), \[40, 70), \[70, 140\]. 

In [ ]:
ax = df.annual_income.plot.hist()
ax.axvline(40, color='black')
ax.axvline(70, color='black')

In [ ]:
ax = df.annual_income.plot.hist()
ax.axvline(40, color='black')
ax.axvline(80, color='black')

In [ ]:
df.head()

In [ ]:
df['income_bin'] = pd.cut(df.annual_income, [0, 50, 80, 140])

plt.figure(figsize=(13, 7))
sns.scatterplot(
    x='age',
    y='spending_score',
    data=df,
    hue='income_bin',
)
plt.title("How does age compare to spending score\nwithin each income bin?")

In [ ]:
df['income_bin'] = pd.cut(df.annual_income, [0, 40, 70, 140])

plt.figure(figsize=(13, 7))
sns.scatterplot(
    x='age',
    y='spending_score',
    data=df,
    hue='income_bin',
)
plt.title("How does age compare to spending score\nwithin each income bin?")

**Takeaways:**

## Summary

- annual income and spending score are good candidates for clustering
- older folks past some cutoff (40+) tend to not high values for spending score
- theres a good number of younger folks (30-) with low incomes and high spending scores
- gender didn't really seem to have an impact